In [1]:
# from golden import *
import pandas as pd
import numpy as np
from pandas_gbq import read_gbq
import pydata_google_auth as pydata

import google.auth
import gspread
from google.auth.transport.requests import Request
from google.auth.credentials import Credentials
from gspread_dataframe import set_with_dataframe
from google.cloud import bigquery

In [2]:
# Regras de Sistema

# Defina sua consulta SQL 
query1 = """
WITH
  base_produto AS (
  SELECT
    pl.id AS id_plano,
    pdt.display_name AS produto
  FROM `curated_health.health_plan` pl
  LEFT JOIN `petlove-dataeng-prod-01.curated_health.health_product_type` pdt
    ON pl.product_type_id=pdt.id
    AND pdt.refdate <= CURRENT_DATE()
  WHERE 1=1
    AND pl.refdate <= CURRENT_DATE()
  ),
  cmv AS (
  SELECT
    cm.id_partner_transfer,
    cm.status_novo,
    cm.vl_bruto,
    cm.numero_atendimento,
    cm.id_atendimento,
    cm.data_origem_date,
    cm.id_pet_bu AS id_pet,
    nome_procedimento,
    ds_grupo_procedimento,
    DATE_TRUNC(cm.data_origem_date, month) AS mes_lancamento,
    pt.under_analysis_reason,
    cm.score_neurotech,
    cm.score_neurotech_faixa,
    status_glosa,
    cm.produto,
    cm.fl_glosa_definitiva,
    cm.nome_procedimento_lancado_prestador,
    cm.portifolio
  FROM `op_health.custos_medicos` cm
  LEFT JOIN `curated_health.health_partner_transfer` pt
    ON cm.id_partner_transfer = pt.id
    AND pt.refdate <= CURRENT_DATE()
  LEFT JOIN base_produto bp
    ON cm.id_plano = bp.id_plano
  WHERE 1=1
    AND cm.status = 'Ativo'
    AND DATE_TRUNC(cm.data_origem_date, month) >= '2025-01-01'
    AND DATE_TRUNC(cm.data_origem_date, month) < '2025-02-01'
  ),
  funil AS (
  SELECT
    id_partner_transfer,
    fl_em_analise_regra,
    fl_glosado,
    fl_auditoria,
    fl_recursado,
    fl_recurso_recusado,
    fl_recurso_aceito
  FROM `op_health.funil_auditoria_analitico`
  ),
  final AS (
  SELECT
    c.id_partner_transfer,
    c.id_atendimento,
    c.portifolio,
    c.status_novo,
    c.under_analysis_reason,
    c.vl_bruto,
    c.mes_lancamento,
    c.data_origem_date,
    c.nome_procedimento,
    f.fl_em_analise_regra,
    f.fl_glosado,
    c.ds_grupo_procedimento AS grupo_procedimento,
    IF (status_glosa IS NULL, 'Não Glosa', status_glosa) AS status_atuacao,
    score_neurotech,
    score_neurotech_faixa,
    produto,
    fl_auditoria,
    fl_recursado,
    fl_recurso_recusado,
    fl_recurso_aceito,
    fl_glosa_definitiva,
    c.numero_atendimento,
    c.nome_procedimento_lancado_prestador
  FROM cmv c
  LEFT JOIN funil f
    ON c.id_partner_transfer = f.id_partner_transfer
  WHERE 1=1
    AND f.fl_em_analise_regra = 1
  ),
  base_classificacao_regra AS (
  SELECT
    *,
    SPLIT(under_analysis_reason, '"')[SAFE_ORDINAL(2)] AS procedimento_1_mensageria,
    SPLIT(under_analysis_reason, '"')[SAFE_ORDINAL(4)] AS procedimento_2_mensageria,
    CASE
      WHEN under_analysis_reason LIKE "%foi usado recentemente%" AND SPLIT(under_analysis_reason, '"')[SAFE_ORDINAL(2)] = SPLIT(under_analysis_reason, '"')[SAFE_ORDINAL(4)] THEN 'Procedimento Ofensor'
      WHEN under_analysis_reason LIKE "%foi usado recentemente%" AND SPLIT(under_analysis_reason, '"')[SAFE_ORDINAL(2)] <> SPLIT(under_analysis_reason, '"')[SAFE_ORDINAL(4)] THEN 'Auto excludente'
      WHEN under_analysis_reason = "Será analisado pela equipe técnica da Petlove, porque este mesmo procedimento foi registrado recentemente para esse pet." THEN 'Recorrência - Sem tempo definido'
      WHEN under_analysis_reason = "recurrency_by_year" THEN 'Recorrência - Anual'
      WHEN under_analysis_reason = "recurrency_by_week" THEN 'Recorrência - Semanal'
      WHEN under_analysis_reason = "recurrency_by_month" THEN 'Recorrência - Mensal'
      WHEN under_analysis_reason = "recurrency_by_day" THEN 'Recorrência - Diária'
      WHEN under_analysis_reason = "recurrency_by_contract" THEN 'Recorrência - Contrato'
      WHEN under_analysis_reason LIKE "%está sendo lançado neste mesmo atendimento%" THEN 'Auto excludente'
      WHEN under_analysis_reason = "Solicitação aceita não finalizada." THEN 'Falta de anexo'
      WHEN under_analysis_reason IS NULL THEN 'Sem motivo especificado'
      ELSE 'N/I' END AS classificacao_motivo_regra
  FROM final
  ),
  base_especificacao_regra AS (
  SELECT
    *,
    CASE
      WHEN classificacao_motivo_regra = 'Sem motivo especificado' THEN 'N/I'
      WHEN classificacao_motivo_regra = 'Procedimento Ofensor' THEN CONCAT(classificacao_motivo_regra, ' | ', procedimento_1_mensageria)
      WHEN classificacao_motivo_regra = 'Auto excludente' THEN CONCAT(classificacao_motivo_regra, '   |   ', procedimento_1_mensageria, ' // ',procedimento_2_mensageria)
      WHEN classificacao_motivo_regra = 'Recorrência - Sem tempo definido' THEN CONCAT(classificacao_motivo_regra, ' | ', produto, ' | ',nome_procedimento_lancado_prestador)
      WHEN classificacao_motivo_regra = 'Recorrência - Anual' THEN CONCAT(classificacao_motivo_regra, ' | ', produto, ' | ',nome_procedimento_lancado_prestador)
      WHEN classificacao_motivo_regra = 'Recorrência - Semanal' THEN CONCAT(classificacao_motivo_regra, ' | ', produto, ' | ',nome_procedimento_lancado_prestador)
      WHEN classificacao_motivo_regra = 'Recorrência - Mensal' THEN CONCAT(classificacao_motivo_regra, ' | ', produto, ' | ',nome_procedimento_lancado_prestador)
      WHEN classificacao_motivo_regra = 'Recorrência - Diária' THEN CONCAT(classificacao_motivo_regra, ' | ', produto, ' | ',nome_procedimento_lancado_prestador)
      WHEN classificacao_motivo_regra = 'Recorrência - Contrato' THEN CONCAT(classificacao_motivo_regra, ' | ', produto, ' | ',nome_procedimento_lancado_prestador)
      WHEN classificacao_motivo_regra = 'Falta de anexo' THEN CONCAT(classificacao_motivo_regra, ' | ', grupo_procedimento)
    END AS regra
  FROM base_classificacao_regra
  ),
  teste AS (
  SELECT
    numero_atendimento,
    id_atendimento,
    portifolio AS portfolio,
    mes_lancamento,
    data_origem_date,
    fl_em_analise_regra,
    fl_auditoria,
    fl_glosado,
    fl_recursado,
    fl_recurso_recusado,
    fl_recurso_aceito,
    fl_glosa_definitiva,
    score_neurotech_faixa,
    produto,
    classificacao_motivo_regra,
    regra,
    vl_bruto
  FROM base_especificacao_regra
  )
SELECT
  *
FROM teste
"""

In [3]:
# Custos

# Defina sua consulta SQL
query2 = """
WITH
  data_pagamento_cte AS (
  SELECT
    DISTINCT id_contabil,
    dt_quitacao AS dt_pagamento,
    dt_solicitacao_nota,
    situacao_pagamento
  FROM `op_health.dts_contabil`
  ),
  final_1 AS (-- Junção entre custos médicos e cte pagamento
  SELECT
    cm.*,
    ben.num_chip,
    CAST(cm.numero_atendimento AS string) num_atendimento_str,
    -- Vamos utilizar como filtro na aba analítica faa.fl_glosado,
    faa.fl_auditoria,
    faa.fl_aprovado_direto,
    faa.fl_recursado,
    faa.fl_em_analise_regra,
    dpc.dt_pagamento,
    dpc.situacao_pagamento,
    dpc.dt_solicitacao_nota,
    DATE_DIFF(dpc.dt_pagamento, cm.data_origem_date, day) AS diff_lancamento_pagamento
  FROM `op_health.custos_medicos` cm -- Agora vamos trazer o microchip DO pet
  LEFT JOIN `op_health.beneficiarios` ben
    ON cm.id_pet_bu = ben.id_pet_bu -- Agora vamos trazer informações sobre auditoria
  LEFT JOIN `op_health.funil_auditoria_analitico` faa
    ON cm.id_atendimento = faa.id_atendimento -- Agora vamos trazer a DATA de pagamento (Para calcular tempo médio de pagamento)
  LEFT JOIN data_pagamento_cte dpc
    ON cm.id_contabil = dpc.id_contabil
  WHERE 1=1
    AND cm.sistema_origem_custo = 'ERP Nofaro'
    AND status = 'Ativo'
    AND DATE_TRUNC(cm.data_origem_date, month) >= '2025-01-01'
    AND DATE_TRUNC(cm.data_origem_date, month) < '2025-02-01'
  ),
  final_2 AS (-- Criação de flag's para saber status de pagamento combinados com status_novo
  SELECT
    f1.*,
    IF ((status_novo = 'Aprovado' OR status_novo = 'Recurso aceito') AND situacao_pagamento IN ('Pago'), 1, 0) AS fl_aprovado_pago,
    IF ((status_novo = 'Aprovado' OR status_novo = 'Recurso aceito') AND (situacao_pagamento IN (
          'Pendente',
          'Recusado',
          'Aguardando nota',
          'Aguardando split') OR situacao_pagamento IS NULL), 1, 0) AS fl_aprovado_nao_pago,
    IF (status_novo IN ('Recurso aceito') AND situacao_pagamento = 'Pago', 1, 0) AS fl_glosado_pago,
    IF (status_novo IN ('Recurso aceito') AND (situacao_pagamento IN (
          'Pendente',
          'Recusado',
          'Aguardando nota',
          'Aguardando split') OR situacao_pagamento IS NULL), 1, 0) AS fl_glosado_nao_pago
  FROM final_1 f1
  )
SELECT
  *
FROM final_2
"""

In [ ]:
# Define o project_id
project_id='petlove-dataeng-prod-01'

# Carregar os dados do BigQuery para um DataFrame do pandas
df_regras = read_gbq(query1, project_id)
df_custos = read_gbq(query2, project_id)

# Exibir os dados
# df.head()

In [ ]:
df_regras.info()

In [ ]:
df_custos.info()

In [7]:
df_result1 = df_custos.merge(df_regras, on='id_atendimento', how='left')

In [ ]:
colunas = list(df_result1.columns)
print(colunas)

In [26]:
colunas_desejadas = [#'empresa',
                     #'empresa_titular',                                      
                     #'nome_empresa',
                     #'sistema_origem_custo',
                     #'portifolio',
                     #'id_contabil',
                     #'id_tutor',
                     #'nome_tutor',
                     #'cpf_tutor',
                     #'cpf_tutor_hash',
                     #'cidade_tutor',
                     #'estado_tutor',
                     #'microrregiao',
                     #'nome_plano',
                     #'nome_plano_dts_pet',
                     'uf_plano',
                     'regiao_plano',
                     #'id_plano',
                     #'cd_contrato',
                     'id_pet_bu',
                     'nome_pet',
                     #'dt_inclusao_pet',
                     #'dt_nascimento_pet',
                     'idade_pet',
                     'especie_pet',
                     #'raca_pet',
                     #'genero_pet',
                     'data_compra',
                     'status',
                     #'id_status_novo',
                     'status_novo',
                     #'mot_canc_guia',
                     #'id_partner_transfer',
                     'id_atendimento',
                     'id_procedimento',
                     'macrogrupo_procedimento',
                     'subgrupo_categoria_procedimento',
                     'ds_grupo_procedimento',
                     #'numero_atendimento_x',
                     #'tipo_impressao_procedimento',
                     #'grupo_carencia',
                     'nome_procedimento',
                     'nome_procedimento_2',
                     'qtd_solicitada_prestador',
                     'data_solicitacao',
                     #'data_origem',
                     'data_origem_date_x',
                     #'data_aviso_prestador',
                     'data_aviso_prestador_date',
                     #'anomes_aviso_prestador',
                     #'prestador_executor_id',
                     'prestador_executor_nome',
                     #'prestador_solicitante_id',
                     'prestador_solicitante_nome',
                     #'data_auditoria',
                     'data_auditoria_date',
                     #'valor_copart',
                     #'local_pgmnto_copart',
                     #'pet_idade_proced',
                     #'fl_compra_adicional',
                     #'tipo_compra_add',
                     'valor_origem',
                     'vl_bruto_x',
                     'valor_autorizado',
                     'valor_autorizado_novo',
                     #'data_glosa',
                     'data_glosa_date',
                     'motivo_glosa',
                     #'motivo_glosa_id',
                     'motivo_glosa_desc',
                     'fl_glosa_definitiva_x',
                     'status_glosa',
                     'valor_glosado',
                     'valor_glosado_novo',
                     'fl_edicao_procedimento',
                     #'id_procedimento_lancado_prestador',
                     #'nome_procedimento_lancado_prestador',
                     #'id_auditor_edicao',
                     'vl_procedimento_anterior_edicao',
                     'diff_apos_edicao',
                     'score_neurotech',
                     'score_neurotech_faixa_x',
                     'produto_x',
                     #'num_chip',
                     #'num_atendimento_str',
                     'fl_auditoria_x',
                     'fl_aprovado_direto',
                     'fl_recursado_x',
                     'fl_em_analise_regra_x',
                     'dt_pagamento',
                     'situacao_pagamento',
                     #'dt_solicitacao_nota',
                     #'diff_lancamento_pagamento',
                     'fl_aprovado_pago',
                     'fl_aprovado_nao_pago',
                     'fl_glosado_pago',
                     'fl_glosado_nao_pago',
                     #'numero_atendimento_y',
                     #'portfolio',
                     #'mes_lancamento',
                     #'data_origem_date_y',
                     'fl_em_analise_regra_y',
                     'fl_auditoria_y',
                     'fl_glosado',
                     'fl_recursado_y',
                     'fl_recurso_recusado',
                     'fl_recurso_aceito',
                     'fl_glosa_definitiva_y',
                     'score_neurotech_faixa_y',
                     #'produto_y',
                     'classificacao_motivo_regra',
                     'regra',
                     'vl_bruto_y']
df_result2 = df_result1[colunas_desejadas]

In [ ]:
df_result3 = df_result2.rename(columns={'data_origem_date_x': 'data_origem_date',
                                        'Nome': 'Cliente',
                                        'Salário': 'Renda'})
df_result3.info()

In [ ]:
def estimate_query_cost(query, project_id):
    """
    Estima o custo da execução de uma query no BigQuery.
    
    Parâmetros:
    - query (str): A query SQL a ser analisada.
    - project_id (str): ID do projeto no Google Cloud.

    Retorna:
    - Tamanho da consulta em GB e o custo estimado.
    """
    client = bigquery.Client(project=project_id)
    
    job_config = bigquery.QueryJobConfig(dry_run=True)  # Ativa o modo dry run
    query_job = client.query(query, job_config=job_config)
    
    # Pegando os bytes que seriam processados
    bytes_processed = query_job.total_bytes_processed
    gb_processed = bytes_processed / (1024 ** 3)  # Convertendo para GB
    
    # Custo estimado (preço padrão: $6 por TB)
    cost_estimate = (gb_processed / 1024) * 6  # Convertendo GB -> TB
    
    print(f"🔹 Dados processados: {gb_processed:.2f} GB")
    print(f"💰 Custo estimado: ${cost_estimate:.4f} (Base: $6/TB)")
    
    return gb_processed, cost_estimate

estimate_query_cost(query, project_id)

In [ ]:
# Definir os escopos necessários (inclui acesso ao Google Cloud, Sheets e Drive)
SCOPES = [
    'https://www.googleapis.com/auth/cloud-platform',
    "https://www.googleapis.com/auth/drive",
    "https://www.googleapis.com/auth/spreadsheets"
]
credentials = pydata.get_user_credentials(
    SCOPES,
    auth_local_webserver=True,
)

# Autoriza o gspread com as novas credenciais
gc = gspread.authorize(credentials)

# Abrir a planilha pelo nome (ou pelo ID)
spreadsheet = gc.open("analise_neurotech")  # Ou use gc.open_by_key("ID_DA_PLANILHA")

# Selecionar a aba desejada
worksheet = spreadsheet.worksheet("Sheet2")  # Ou use spreadsheet.sheet1 para a primeira aba

# Enviar os dados para o Google Sheets
set_with_dataframe(worksheet, df)

print("Dados exportados com sucesso!")

------------------------